In [39]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import SniperUtils as SU
import os.path
from src.core import IterUtils
from src.view import DataView as dview
from src.PCapLocationParams import PCapLocationParams
from src.core.PostgresConnection import PostgresConnection
from src.MarketDate import *

pd.options.display.width = 920
pd.options.display.max_columns = 500
pd.options.display.max_rows = 220
pd.options.display.max_rows = 263

market_dates = get_feather_dates(340, name = 'synthetic_book_2', month=[5,6])
short_mkt_ts = [pd.Timestamp(d) for d in market_dates[0:5]]
synth_books = SU.load_synth_book_feather(channel = 340, name = 'synthetic_book_2', market_dates=market_dates)
sb_date_grps = synth_books.loc[:,2,:].groupby(level=0)
root_df = SU.load_sniper_opps('zs')

In [2]:
import numpy as np
sroot_df = root_df.copy().drop(columns=[
    'exit_pbook', 'entry_ev', 'entry_pbook', 'entry_pbook_dir', 'entry_leg_books',
    'exit_event', 'exit_pbook', 'exit_pbook_dir', 'exit_leg_books',
    'cp1_win', 'cp1_pbook', 'cp1_pbook_dir', 'cp2_win', 'cp2_pbook', 'cp2_pbook_dir', 't_time'])
sroot_df = sroot_df.loc[sroot_df.index.get_level_values(0) > '2021-05-02']
sroot_df = sroot_df.sort_values(by=['market_date'])
sroot_df = sroot_df.loc[:,'+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)',sroot_df['entry_ticks'] > 1,:]
sroot_df = sroot_df.reset_index(level=1, drop=True)
sdict = dview.get_style_dict(sroot_df)
sdict.update({'entry_pnl': '{:.2f}'})
sdict.update({'fsn_win': lambda x: x.microseconds, 'lsn_win': lambda x: x.microseconds})
entry_ticks_hm = dview.Heatmap(steps={
    pd.Timedelta('0us'): dview.Colors.blue(),
    pd.Timedelta('50us'): dview.Colors.purple(), 
    pd.Timedelta('100us'): dview.Colors.pink(),
    pd.Timedelta('1000us'): dview.Colors.green()},interpolate=False)
formatters = {'lsn_win': entry_ticks_hm.to_lambda()}
def highlight_max(x):
    if x.name in formatters:
        return x.apply(formatters[x.name])
#     if pd.api.types.is_string_dtype(x):
#         return x.apply(lambda s: f'background-color: green')
#     if pd.api.types.is_numeric_dtype(x):
#         return x.apply(lambda s: f'background-color: blue' if s > 4 else 'background-color: purple')
    def get_col(i):
        return dview.Color(44,44,44) if i % 2 == 1 else dview.Color(66,66,66)
    return [f'background-color: {get_col(i)}' for i in range(1, len(x) +1)]
#     return x.apply(lambda s: f'background-color: black')
#     return np.where(s != '5', props, '')
sroot_df.loc[short_mkt_ts,:].style.format(sdict).apply(highlight_max, axis=0)
# .apply({'entry_ticks': lambda x: f'background-color: blue' })
# format(sdict)

In [51]:
def add_shot_snapshot_rows(df_grps: pd.DataFrame, sb_date_grps, delay: pd.Timedelta, show_all_changes = True) -> pd.DataFrame:
    def merge_synth_books(x):
        # Get the last book state before event 'eid' 
        synth_book = sb_date_grps.get_group(pd.Timestamp(x.name[0]))
        sb_df = synth_book.reset_index(level='eid')
        sb_pre_df = sb_df.loc[sb_df['t_time'] <= pd.Timestamp(x.name[1])]
        sb_start = sb_pre_df.max()
        sb_start_eid = sb_start['eid']
        sb_start_time = sb_start['t_time']
        sb_stop_time = sb_start_time + delay
        print(sb_start)
#         res = sb_df.loc[(sb_df['eid'] >= sb_start_eid) & (sb_df['t_time'] <= sb_stop_time)]
        res = sb_df.loc[(sb_df['eid'] >= sb_start_eid)]
        return res

    ops = df_grps.apply(merge_synth_books)
#     print('idx len ' + str(ops.index))
#     ops.index.set_names(['market_date', 'start_eid'], inplace=True)
#     print(type(ops))
#     ops.rename(columns={'id': 'pid'}, inplace=True)
    return ops

df = sroot_df.copy()
# sb_date_grps = synth_books.groupby(level=[0,1])
df = df.loc[short_mkt_ts,:]
# df.loc[short_mkt_ts,:].style.format(sdict).apply(highlight_max, axis=0)
# sbg = sb_date_grps.get_group(('2021-05-02', 2))
# sb_date_grps.apply(lambda g: g)
# sbg.loc[]
# df
opp_spec_root = add_shot_snapshot_rows(df.groupby(level=[0,1]), sb_date_grps, pd.Timedelta('175us'))
opp_spec_root

eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_p_1    NaN
ask_q_1    NaN
dtype: object
eid        NaN
t_time     NaT
bid_p_1    NaN
bid_q_1    NaN
ask_

,,,eid,t_time,symbol,bid_p_1,bid_q_1,ask_p_1,ask_q_1
market_date,eid,market_date,,,,,,,


In [32]:
# for b in sb_date_grps:
#     print(b)
#     break
print(opp_spec_root[0])

1


In [16]:
sbg = sb_date_grps.get_group('2021-05-02')
sbg.loc[:, 2, :]

t_time                                             symbol  bid_p_1  bid_q_1  ask_p_1  ask_q_1
market_date eid                                                                                                                          
2021-05-02  0        2021-05-02 21:01:31.201258175  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -3.25        2     6.00        1
            1042     2021-05-02 21:07:44.289517791  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -3.25        2      NaN        0
            6330     2021-05-02 23:50:25.337698639  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -3.25        1      NaN        0
            7031     2021-05-02 23:55:00.073015161  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -3.00        1      NaN        0
            7495     2021-05-02 23:55:00.434013531  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -3.00        1     1.50        1
...                                            ...                                                ...      ...      ...      ...      ...
            26956505 2021-05-03 18:20:30.000113227  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -2.50        1     1.25        1
            26956717 2021-05-03 18:20:30.000143691  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -2.75        1     1.25        1
            26957139 2021-05-03 18:20:30.000204497  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -2.75        1     1.50        1
            26957372 2021-05-03 18:20:30.000238239  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -2.75        1     1.75        1
            26961069 2021-05-03 18:30:00.071391477  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -2.75        1     3.00        1

[70165 rows x 6 columns]

In [3]:
df = sroot_df.copy()
df = df.loc[short_mkt_ts,:]
opp_spec_root = SU.add_shot_snapshot_rows(df.groupby(level=[0,1]), sb_date_grps, pd.Timedelta('175us'))

7182897


IndexError: list index out of range

In [96]:
secs = pd.read_feather(PCapLocationParams(340, '2021-05-02').feather_file_path('security'))
syn_leg_bks = pd.read_feather(PCapLocationParams(340, '2021-05-02').feather_file_path('synthetic_leg_book_2'))
syn_lbs = syn_leg_bks.copy()
syn_lbs = syn_lbs.loc[(syn_lbs['eid'] >= 4706500) & (syn_lbs['eid'] <= 4706953)]
def get_latest(x):
    return x.iloc[-1]
syn_lbs = pd.merge(syn_lbs, secs.rename(columns={'sid': 'sec_id'})[['sec_id', 'symbol']], left_on='sid', right_on='sec_id').drop(columns=['sec_id'])
syn_lbs.groupby('sid').apply(get_latest)[['eid', 't_time', 'sid', 'symbol', 'bid_p_1', 'bid_q_1', 'ask_p_1', 'ask_q_1']]
# syn_leg_bks.loc[syn_leg_bks['bid_p_1'] < syn_leg_bks['ibid_p_1']]

,eid,t_time,sid,symbol,bid_p_1,bid_q_1,ask_p_1,ask_q_1
sid,,,,,,,,
12059,4706953,2021-05-03 05:57:38.243458845,12059,ZSN1-ZSX1,193.00,10,193.25,1
19321,4706509,2021-05-03 05:57:33.707545005,19321,ZSQ1-ZSU1,89.25,1,89.75,4
20350,4706512,2021-05-03 05:57:33.707606063,20350,ZSU1-ZSX1,41.75,1,42.00,1
24491,4706944,2021-05-03 05:57:38.216950767,24491,ZSN1-ZSQ1,62.50,6,62.75,8


In [91]:
def merge_syn_bk_cols(a, b, direc='forward'):
    return pd.merge_asof(a, b, on='time', direction=direc)

a = pd.DataFrame([[pd.Timedelta('1s'), 5], [pd.Timedelta('5s'), 6]], columns=['time', 'val'])
b = pd.DataFrame([[pd.Timedelta('1s'), 500], [pd.Timedelta('2s'), 550], [pd.Timedelta('6s'), 600]], columns=['time', 'val'])
# display(merge_syn_bk_cols(a, b))
# display(merge_syn_bk_cols(a, b, 'backward'))
# display(merge_syn_bk_cols(b, a))
# display(merge_syn_bk_cols(b, a, 'backward'))
b.iloc[-2:-1]
    

,time,val
1,0 days 00:00:02,550


In [82]:
secs = pd.read_feather(PCapLocationParams(340, '2021-05-02').feather_file_path('security'))
syn_leg_bks = pd.read_feather(PCapLocationParams(340, '2021-05-02').feather_file_path('book_2021-07-09_18-12'))
syn_lbs = syn_leg_bks.copy()
syn_lbs = syn_lbs.loc[(syn_lbs['eid'] >= 4706509) & (syn_lbs['eid'] <= 4706953)]
def get_latest(x):
    return x.iloc[-1]
syn_lbs = pd.merge(syn_lbs, secs.rename(columns={'sid': 'sec_id'})[['sec_id', 'symbol']], left_on='sid', right_on='sec_id').drop(columns=['sec_id'])
syn_lbs.groupby('sid').apply(get_latest)[['eid', 't_time', 'sid', 'symbol', 'bid_p_1', 'ask_p_1']]
# syn_leg_bks.loc[syn_leg_bks['bid_p_1'] < syn_leg_bks['ibid_p_1']]

,eid,t_time,sid,symbol,bid_p_1,ask_p_1
sid,,,,,,
12059,4706953,2021-05-03 05:57:38.243458845,12059,ZSN1-ZSX1,193.00,193.25
19321,4706509,2021-05-03 05:57:33.707545005,19321,ZSQ1-ZSU1,89.25,89.75
20350,4706512,2021-05-03 05:57:33.707606063,20350,ZSU1-ZSX1,41.75,42.00
24491,4706944,2021-05-03 05:57:38.216950767,24491,ZSN1-ZSQ1,62.50,62.75


In [79]:
print(62.50 + 89.25 - 193.25 + 41.75)
print(62.75 + 89.75 - 193.00 + 42.00)

0.25
1.5


In [9]:
sb_ = pd.read_feather(PCapLocationParams(340, '2021-05-02').feather_file_path('synthetic_book_2'))
sb_.loc[[sb_['eid'].idxmax()]]
sb_ = sb_.loc[sb_['eid'] == 4706953]
sb_

,eid,t_time,id,symbol,bid_p_1,bid_q_1,bid_p_2,bid_q_2,bid_p_3,bid_q_3,ask_p_1,ask_q_1,ask_p_2,ask_q_2,ask_p_3,ask_q_3
33859,4706953,2021-05-03 05:57:38.243458845,2,+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...,0.25,1,-0.75,1,-1.0,1,1.5,1,1.75,3,2.0,4


In [8]:
opp_spec = opp_spec_root.copy()
opp_spec.loc[opp_spec['market_date'] == '2021-05-02']

eid                        t_time  pid                                             symbol  bid_p_1  bid_q_1  ask_p_1  ask_q_1  bid_change  ask_change market_date
market_date start_eid row_id                                                                                                                                                                        
2021-05-02  29519     479        29519 2021-05-03 00:00:01.722108603    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        1     3.50        1        True       False  2021-05-02
            29788     499        29788 2021-05-03 00:00:01.758525223    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        1     3.50        1        True       False  2021-05-02
                      501        29793 2021-05-03 00:00:01.758670651    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.50        6     3.50        1        True       False  2021-05-02
            4706953   33858    4706944 2021-05-03 05:57:38.216950767    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        1        True        True  2021-05-02
            10368010  86092   10368010 2021-05-03 13:37:27.306736671    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.50        1    -0.25        1       False        True  2021-05-02
            12910850  121239  12910843 2021-05-03 14:03:54.019289777    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.50        2     0.25        1       False        True  2021-05-02
            26923577  242981  26923187 2021-05-03 18:19:20.652375095    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.00        3     0.00        1        True        True  2021-05-02

In [81]:
opp_spec = opp_spec_root.copy()
opp_spec.head(220)

eid                        t_time  pid                                             symbol  bid_p_1  bid_q_1  ask_p_1  ask_q_1  bid_change  ask_change market_date
market_date start_eid row_id                                                                                                                                                                        
2021-06-24  15501995  91372   15501857 2021-06-25 16:04:53.854291589    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.00        7     0.00        1        True        True  2021-06-24
            16187175  94701   16187175 2021-06-25 16:31:31.796643975    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        2     1.25        3        True       False  2021-06-24
            16727763  96921   16727763 2021-06-25 16:52:54.986991399    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        2     1.25        9        True       False  2021-06-24
                      96923   16727784 2021-06-25 16:52:54.987118117    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        3     1.25        9       False        True  2021-06-24
            17769768  101300  17769643 2021-06-25 17:31:16.759735259    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        5     1.00       15        True        True  2021-06-24
            17892992  102252  17892989 2021-06-25 17:36:14.735689357    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        4     1.50        7        True       False  2021-06-24
                      102254  17893022 2021-06-25 17:36:14.735796453    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        3     1.50        7        True        True  2021-06-24
            18705990  105979  18705990 2021-06-25 18:02:30.565985791    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25       16     1.25       11        True        True  2021-06-24
            18839968  106716  18839968 2021-06-25 18:04:18.040912931    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25       38     1.25       16        True        True  2021-06-24
            19324722  110667  19324722 2021-06-25 18:11:44.033352685    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        2     1.25        2        True       False  2021-06-24

In [7]:
opp_spec = opp_spec_root.copy()
opp_spec.head(220)

eid                        t_time  pid                                             symbol  bid_p_1  bid_q_1  ask_p_1  ask_q_1  bid_change  ask_change market_date
market_date start_eid row_id                                                                                                                                                                        
2021-04-29  1168525   2917     1168525 2021-04-30 01:17:16.383741535    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.00        1     0.25        1        True       False  2021-04-29
            5968738   12838    5968592 2021-04-30 13:31:00.194385149    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     2.00        1        True       False  2021-04-29
            6445121   14098    6445121 2021-04-30 13:33:07.248090691    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.75        2     0.50        1        True       False  2021-04-29
            19136020  51212   19136020 2021-04-30 18:13:50.999213711    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.50        2    -0.25        2       False        True  2021-04-29
2021-05-02  29519     1158       29518 2021-05-03 00:00:01.722106509    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     3.50        1        True       False  2021-05-02
            29788     1214       29788 2021-05-03 00:00:01.758525223    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        1     3.50        1        True       False  2021-05-02
                      1216       29793 2021-05-03 00:00:01.758670651    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.50        6     3.50        1        True       False  2021-05-02
            4706953   11816    4700650 2021-05-03 05:56:33.428939865    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        1        True        True  2021-05-02
            10368010  24840   10368010 2021-05-03 13:37:27.306736671    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.75        2     0.25        1       False        True  2021-05-02
            12910850  31593   12910843 2021-05-03 14:03:54.019289777    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.75        1     0.25        1       False        True  2021-05-02
            26923577  58751   26923187 2021-05-03 18:19:20.652375095    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.00        2     0.00        1        True        True  2021-05-02
2021-05-03  7182897   14691    7181155 2021-05-04 13:31:56.788235195    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.50        1     1.00        1        True        True  2021-05-03
            7250526   14845    7250242 2021-05-04 13:32:19.515821047    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.50        1     1.00        1       False        True  2021-05-03
            11264349  21592   11264349 2021-05-04 14:11:21.918492235    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     1.25        4       False        True  2021-05-03
            14200881  28898   14200874 2021-05-04 14:58:33.609011949    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        5     1.25        2        True       False  2021-05-03
            17248005  35834   17247989 2021-05-04 16:10:38.388389773    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        2        True       False  2021-05-03
            17308444  36221   17308431 2021-05-04 16:11:31.796650655    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        3     1.00        1        True        True  2021-05-03
            17308892  36240   17308882 2021-05-04 16:11:31.833867893    2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        3     1.25        1        True       False  2021-05-03
            17309334  36264   17309334 2021-05-04 16:11:31.896765255    2  +(ZSN1-ZSQ1) +(ZSQ1-

In [13]:
synth_book_map = {}
for date in market_dates:
    synth_book_map[date] = SU.load_synth_book_feather(340, date, 'synthetic_book_2')

In [18]:
syn_bk = synth_book_map['2021-05-02'].copy()
syn_bk = syn_bk.loc[(syn_bk['eid'] >= 1168500) & (syn_bk['eid'] <= 1168700)]
syn_bk['t_tod'] = syn_bk['t_time'] - pd.Timestamp('2021-05-03')
syn_bk['t_tod_us'] = syn_bk['t_tod'].dt.microseconds
sdict = dview.get_style_dict(syn_bk)
for level in range(1, 4):
    for side in ['bid', 'ask']:
        for prop in ['p', 'q']:
            sdict[f'{side}_{prop}_{level}'] = '{:.2f}'
syn_bk.style.format(sdict)

,eid,t_time,id,symbol,bid_p_1,bid_q_1,bid_p_2,bid_q_2,bid_p_3,bid_q_3,ask_p_1,ask_q_1,ask_p_2,ask_q_2,ask_p_3,ask_q_3,t_tod,t_tod_us
6284,1168632,,2,+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),-1.00,1.00,-1.25,4.00,-1.50,1.00,0.50,2.00,1.25,2.00,1.75,2.00,14:32.99,996450


In [16]:
syn_leg_bks = pd.read_feather('/home/john/spartan_store/pcaps/340/2021-05-02_340_24h_feather/synthetic_leg_book_2.feather')

In [17]:
secs = pd.read_feather('/home/john/spartan_store/pcaps/340/2021-05-02_340_24h_feather/security.feather')
secs

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id
0,296835,TRFZ1-TRFH2,2021-12-31 19:20:00,SPREAD,FUT,LR,TRF,True,F,0.010,0.0,50.000,0.0,2,70
1,214428,MKCK1-MKCN3,2021-05-14 17:01:00,SPREAD,FUT,MH,MKC,True,K,1.000,10.0,0.125,0.0,2,70
2,727762,ZMQ1,2021-08-13 17:01:00,FUTURE,FUT,ZS,ZM,True,K,0.100,100.0,1.000,1.0,0,70
3,35930,ZCK2-ZCN2,2022-05-13 17:01:00,SPREAD,FUT,ZC,ZC,True,K,1.000,50.0,0.250,0.0,2,70
4,168192,ZLZ1-ZLK3,2021-12-14 18:01:00,SPREAD,FUT,ZS,ZL,True,K,0.010,600.0,1.000,0.0,2,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,128333,ZMZ1-ZMU3,2021-12-14 18:01:00,SPREAD,FUT,ZS,ZM,True,K,0.100,100.0,1.000,0.0,2,70
3870,784556,MKCH2-MKCN3,2022-03-14 17:01:00,SPREAD,FUT,MH,MKC,True,K,1.000,10.0,0.125,0.0,2,70
3871,6624,EHK1-EHZ3,2021-05-05 17:01:00,SPREAD,FUT,EH,EH,True,K,0.001,29000.0,1.000,0.0,2,70
3872,179582,UWFQ1-UWFF2,2021-08-31 15:30:00,SPREAD,FUT,WP,UWF,True,F,0.010,0.0,25.000,0.0,2,70


In [19]:
bk = pd.read_feather('/home/john/spartan_store/pcaps/340/2021-04-29_340_24h_feather/book.feather')
bk = bk.loc[(bk['eid'] >= 1168500) & (bk['eid'] <= 1168600)]
bk['bid_change'] = bk['bid_p_1'] != bk['bid_p_1'].shift(-1)
bk['ask_change'] = bk['ask_p_1'] != bk['ask_p_1'].shift(-1)
bk = bk.loc[bk['bid_change'] | bk['ask_change']]
bk = bk[['eid', 't_time', 'sid', 'bid_p_1', 'ask_p_1']].copy()
bk = pd.merge(bk, secs.rename(columns={'sid': 'sec_sid'})[['sec_sid', 'symbol', 'sec_type', 'cme_sec_type', 'sec_group', 'asset']], left_on='sid', right_on='sec_sid').drop(columns='sec_sid')
bkg = bk.groupby('sid')
for g in bkg.groups:
    display(bkg.get_group(g))

,eid,t_time,sid,bid_p_1,ask_p_1,symbol,sec_type,cme_sec_type,sec_group,asset
2,1168525,2021-04-30 01:17:16.383741535,12059,182.5,183.25,ZSN1-ZSX1,SPREAD,FUT,ZS,ZS
3,1168541,2021-04-30 01:17:16.384507813,12059,182.5,183.25,ZSN1-ZSX1,SPREAD,FUT,ZS,ZS
4,1168592,2021-04-30 01:17:16.384784297,12059,182.5,183.25,ZSN1-ZSX1,SPREAD,FUT,ZS,ZS
5,1168599,2021-04-30 01:17:16.384796635,12059,182.5,183.25,ZSN1-ZSX1,SPREAD,FUT,ZS,ZS


,eid,t_time,sid,bid_p_1,ask_p_1,symbol,sec_type,cme_sec_type,sec_group,asset
6,1168525,2021-04-30 01:17:16.383741535,19321,87.0,87.50,ZSQ1-ZSU1,SPREAD,FUT,ZS,ZS
7,1168594,2021-04-30 01:17:16.384787313,19321,87.0,87.75,ZSQ1-ZSU1,SPREAD,FUT,ZS,ZS


,eid,t_time,sid,bid_p_1,ask_p_1,symbol,sec_type,cme_sec_type,sec_group,asset
8,1168525,2021-04-30 01:17:16.383741535,20350,37.25,37.5,ZSU1-ZSX1,SPREAD,FUT,ZS,ZS
9,1168594,2021-04-30 01:17:16.384787313,20350,37.25,37.5,ZSU1-ZSX1,SPREAD,FUT,ZS,ZS


,eid,t_time,sid,bid_p_1,ask_p_1,symbol,sec_type,cme_sec_type,sec_group,asset
0,1168525,2021-04-30 01:17:16.383741535,24491,57.5,57.75,ZSN1-ZSQ1,SPREAD,FUT,ZS,ZS
1,1168542,2021-04-30 01:17:16.384508081,24491,57.5,57.75,ZSN1-ZSQ1,SPREAD,FUT,ZS,ZS


In [43]:
slb = syn_leg_bks.copy()
slb = slb.loc[(slb['eid'] >= 1168500) & (slb['eid'] <= 1168700)]
slb = pd.merge(slb, secs.rename(columns={'sid': 'sec_sid'})[['sec_sid', 'symbol', 'sec_type', 'cme_sec_type', 'sec_group', 'asset']], left_on='sid', right_on='sec_sid').drop(columns='sec_sid')
slb['t_tod'] = slb['t_time'] - pd.Timestamp('2021-04-30')
slb['t_tod_us'] = slb['t_tod'].dt.microseconds
sdict = dview.get_style_dict(slb)
tmi_bk_cols = ['implied_matching', 'sec_type', 'asset', 't_time']
for level in range(1, 4):
    for side in ['bid', 'ask']:
        for prop in ['p', 'q']:
            sdict[f'{side}_{prop}_{level}'] = '{:.2f}'
            if level < 3:
                tmi_bk_cols.append(f'i{side}_{prop}_{level}')
            if level > 1:
                tmi_bk_cols.append(f'{side}_{prop}_{level}')
                if prop == 'q':
                    tmi_bk_cols.append(f'd{side}_{prop}_{level}')
slbg = slb.groupby('sid')
for grp in slbg.groups:
    display(slbg.get_group(grp).drop(columns=tmi_bk_cols).style.format(sdict))

,eid,sid,status,bid_p_1,bid_q_1,dbid_q_1,bid_no_1,bid_no_2,bid_no_3,ask_p_1,ask_q_1,dask_q_1,ask_no_1,ask_no_2,ask_no_3,symbol,cme_sec_type,sec_group,t_tod,t_tod_us
5,1168525,12059,READY_TO_TRADE,182.50,2.00,2,2,3,2,183.25,5.00,5,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,383741
6,1168527,12059,READY_TO_TRADE,182.50,2.00,2,2,3,2,183.25,5.00,5,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384459
7,1168541,12059,READY_TO_TRADE,182.50,2.00,2,2,3,2,183.25,5.00,5,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384507
8,1168549,12059,READY_TO_TRADE,182.50,2.00,2,2,3,2,183.25,5.00,5,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384740
9,1168551,12059,READY_TO_TRADE,182.50,2.00,2,2,3,2,183.25,5.00,5,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384741
10,1168554,12059,READY_TO_TRADE,182.50,2.00,2,2,3,2,183.25,5.00,5,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384744
11,1168576,12059,READY_TO_TRADE,182.50,2.00,2,2,3,2,183.25,4.00,4,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384761
12,1168592,12059,READY_TO_TRADE,182.50,1.00,1,1,3,2,183.25,4.00,4,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384784
13,1168599,12059,READY_TO_TRADE,182.50,1.00,1,1,3,2,183.25,4.00,4,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384796
14,1168602,12059,READY_TO_TRADE,182.50,1.00,1,1,3,2,183.25,4.00,4,4,4,5,ZSN1-ZSX1,FUT,ZS,01:17:16.38,384802


,eid,sid,status,bid_p_1,bid_q_1,dbid_q_1,bid_no_1,bid_no_2,bid_no_3,ask_p_1,ask_q_1,dask_q_1,ask_no_1,ask_no_2,ask_no_3,symbol,cme_sec_type,sec_group,t_tod,t_tod_us
21,1168525,19321,READY_TO_TRADE,87.00,3.00,3,3,6,8,87.50,1.00,1,1,5,6,ZSQ1-ZSU1,FUT,ZS,01:17:16.38,383741
22,1168594,19321,READY_TO_TRADE,87.00,3.00,3,3,6,8,87.75,6.00,6,5,6,7,ZSQ1-ZSU1,FUT,ZS,01:17:16.38,384787
23,1168612,19321,READY_TO_TRADE,87.00,3.00,3,3,6,8,87.75,6.00,6,5,6,7,ZSQ1-ZSU1,FUT,ZS,01:17:16.38,384814
24,1168672,19321,READY_TO_TRADE,87.00,3.00,3,3,6,8,87.75,6.00,6,5,6,7,ZSQ1-ZSU1,FUT,ZS,01:17:16.38,384905
25,1168698,19321,READY_TO_TRADE,87.00,3.00,3,3,6,8,87.75,6.00,6,5,6,7,ZSQ1-ZSU1,FUT,ZS,01:17:16.38,384940
26,1168699,19321,READY_TO_TRADE,87.00,3.00,3,3,6,8,87.75,6.00,6,5,6,7,ZSQ1-ZSU1,FUT,ZS,01:17:16.38,384941


,eid,sid,status,bid_p_1,bid_q_1,dbid_q_1,bid_no_1,bid_no_2,bid_no_3,ask_p_1,ask_q_1,dask_q_1,ask_no_1,ask_no_2,ask_no_3,symbol,cme_sec_type,sec_group,t_tod,t_tod_us
27,1168525,20350,READY_TO_TRADE,37.25,1.00,1,1,3,2,37.50,1.00,1,1,4,3,ZSU1-ZSX1,FUT,ZS,01:17:16.38,383741
28,1168594,20350,READY_TO_TRADE,37.25,1.00,1,1,3,2,37.50,1.00,1,1,4,3,ZSU1-ZSX1,FUT,ZS,01:17:16.38,384787
29,1168632,20350,READY_TO_TRADE,37.25,1.00,1,1,3,2,37.75,4.00,4,4,3,2,ZSU1-ZSX1,FUT,ZS,01:17:16.38,384884
30,1168672,20350,READY_TO_TRADE,37.50,1.00,1,1,1,3,37.75,4.00,4,4,3,2,ZSU1-ZSX1,FUT,ZS,01:17:16.38,384905
31,1168673,20350,READY_TO_TRADE,37.50,1.00,1,1,1,3,37.75,4.00,4,4,3,2,ZSU1-ZSX1,FUT,ZS,01:17:16.38,384908


,eid,sid,status,bid_p_1,bid_q_1,dbid_q_1,bid_no_1,bid_no_2,bid_no_3,ask_p_1,ask_q_1,dask_q_1,ask_no_1,ask_no_2,ask_no_3,symbol,cme_sec_type,sec_group,t_tod,t_tod_us
0,1168525,24491,READY_TO_TRADE,57.50,15.00,15,6,6,8,57.75,1.00,1,1,5,5,ZSN1-ZSQ1,FUT,ZS,01:17:16.38,383741
1,1168542,24491,READY_TO_TRADE,57.50,15.00,15,6,6,8,57.75,1.00,1,1,5,5,ZSN1-ZSQ1,FUT,ZS,01:17:16.38,384508
2,1168612,24491,READY_TO_TRADE,57.50,15.00,15,6,6,8,58.00,15.00,15,5,5,4,ZSN1-ZSQ1,FUT,ZS,01:17:16.38,384814
3,1168698,24491,READY_TO_TRADE,57.50,15.00,15,6,6,8,58.00,15.00,15,5,5,4,ZSN1-ZSQ1,FUT,ZS,01:17:16.38,384940
4,1168699,24491,READY_TO_TRADE,57.50,15.00,15,6,6,8,58.00,15.00,15,5,5,4,ZSN1-ZSQ1,FUT,ZS,01:17:16.38,384941


In [46]:
print(57.75 + 87.5 - 182.5 + 37.5)
print(57.5 + 87 - 183.25 + 37.25)

0.25
-1.5


In [37]:
opp_spec = opp_spec_root.copy()
# for k in opp_spec.groupby(level=[0,1]).groups:
#     break
opp_spec.head(220)
# opp_spec.style.format(dview.get_style_dict(opp_spec))

eid                        t_time  id                                             symbol  bid_p_1  bid_q_1  ask_p_1  ask_q_1  bid_change  ask_change market_date
market_date eid                                                                                                                                                                                   
2021-04-29  1168525  2917     1168525 2021-04-30 01:17:16.383741535   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.00        1     0.25        1        True       False  2021-04-29
            5968738  12838    5968592 2021-04-30 13:31:00.194385149   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     2.00        1        True       False  2021-04-29
            6445121  14098    6445121 2021-04-30 13:33:07.248090691   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.75        2     0.50        1        True       False  2021-04-29
            19136020 51212   19136020 2021-04-30 18:13:50.999213711   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.50        2    -0.25        2       False        True  2021-04-29
2021-05-02  29519    1158       29518 2021-05-03 00:00:01.722106509   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     3.50        1        True       False  2021-05-02
            29788    1214       29788 2021-05-03 00:00:01.758525223   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        1     3.50        1        True       False  2021-05-02
                     1216       29793 2021-05-03 00:00:01.758670651   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.50        6     3.50        1        True       False  2021-05-02
            4706953  11816    4700650 2021-05-03 05:56:33.428939865   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        1        True        True  2021-05-02
            10368010 24840   10368010 2021-05-03 13:37:27.306736671   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.75        2     0.25        1       False        True  2021-05-02
            12910850 31593   12910843 2021-05-03 14:03:54.019289777   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.75        1     0.25        1       False        True  2021-05-02
            26923577 58751   26923187 2021-05-03 18:19:20.652375095   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.00        2     0.00        1        True        True  2021-05-02
2021-05-03  7182897  14691    7181155 2021-05-04 13:31:56.788235195   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.50        1     1.00        1        True        True  2021-05-03
            7250526  14845    7250242 2021-05-04 13:32:19.515821047   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.50        1     1.00        1       False        True  2021-05-03
            11264349 21592   11264349 2021-05-04 14:11:21.918492235   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     1.25        4       False        True  2021-05-03
            14200881 28898   14200874 2021-05-04 14:58:33.609011949   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        5     1.25        2        True       False  2021-05-03
            17248005 35834   17247989 2021-05-04 16:10:38.388389773   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        2        True       False  2021-05-03
            17308444 36221   17308431 2021-05-04 16:11:31.796650655   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        3     1.00        1        True        True  2021-05-03
            17308892 36240   17308882 2021-05-04 16:11:31.833867893   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        3     1.25        1        True       False  2021-05-03
            17309334 36264   17309334 2021-05-04 16:11:31.896765255   2  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00   

In [13]:
opp_spec_root

eid                        t_time  pid                                             symbol  bid_p_1  bid_q_1  ask_p_1  ask_q_1  bid_change  ask_change market_date
market_date eid                                                                                                                                                                                    
2021-05-02  29519    1158       29518 2021-05-03 00:00:01.722106509    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     3.50        1        True       False  2021-05-02
            29788    1214       29788 2021-05-03 00:00:01.758525223    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.25        1     3.50        1        True       False  2021-05-02
                     1216       29793 2021-05-03 00:00:01.758670651    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.50        6     3.50        1        True       False  2021-05-02
            4706953  11816    4700650 2021-05-03 05:56:33.428939865    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        1        True        True  2021-05-02
            10368010 24840   10368010 2021-05-03 13:37:27.306736671    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.75        2     0.25        1       False        True  2021-05-02
            12910850 31593   12910843 2021-05-03 14:03:54.019289777    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.75        1     0.25        1       False        True  2021-05-02
            26923577 58751   26923187 2021-05-03 18:19:20.652375095    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -1.00        2     0.00        1        True        True  2021-05-02
2021-05-03  7182897  14691    7181155 2021-05-04 13:31:56.788235195    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.50        1     1.00        1        True        True  2021-05-03
            7250526  14845    7250242 2021-05-04 13:32:19.515821047    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.50        1     1.00        1       False        True  2021-05-03
            11264349 21592   11264349 2021-05-04 14:11:21.918492235    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     1.25        4       False        True  2021-05-03
            14200881 28898   14200874 2021-05-04 14:58:33.609011949    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        5     1.25        2        True       False  2021-05-03
            17248005 35834   17247989 2021-05-04 16:10:38.388389773    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        2        True       False  2021-05-03
            17308444 36221   17308431 2021-05-04 16:11:31.796650655    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        3     1.00        1        True        True  2021-05-03
            17308892 36240   17308882 2021-05-04 16:11:31.833867893    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        3     1.25        1        True       False  2021-05-03
            17309334 36264   17309334 2021-05-04 16:11:31.896765255    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...     0.00        1     1.25        1        True       False  2021-05-03
            19152907 42674   19152907 2021-05-04 17:08:32.736380049    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.50        6        True       False  2021-05-03
            19154676 42712   19154676 2021-05-04 17:08:33.010432475    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        2        True       False  2021-05-03
            19154979 42718   19154979 2021-05-04 17:08:33.059341445    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-...    -0.25        1     1.25        3       False        True  2021-05-03
            20523097 46608   20523097 2021-05-04 17:48:47.523398467    0  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +

In [132]:
opp_spec = opp_spec_root.copy()
opp_spec['t_tod'] = opp_spec['t_time'] - (opp_spec['market_date'] + pd.Timedelta('1d'))
# opp_spec = opp_spec.drop(columns=['market_date'])
# opp_spec_display_cols = list(opp_spec.columns)
# opp_spec_display_cols.insert(0, opp_spec_display_cols.pop(len(opp_spec_display_cols) - 1))
# opp_spec = opp_spec.reindex(columns=opp_spec_display_cols)
opp_spec_style_dict = dview.get_style_dict(opp_spec)
opp_spec_style_dict['bid_p_1'] = '{:.2f}'
opp_spec_style_dict['ask_p_1'] = '{:.2f}'
opp_spec_style_dict['eid'] = '{}'
# opp_spec
# opp_spec.set_index(opp_spec.index).style.format(dview.get_style_dict(opp_spec)).hide_columns(['t_time', 'symbol', 'market_date'])

MultiIndex([('2021-05-02',    29519,   1158),
            ('2021-05-02',    29788,   1214),
            ('2021-05-02',    29788,   1216),
            ('2021-05-02',  4706953,  11816),
            ('2021-05-02', 10368010,  24840),
            ('2021-05-02', 12910850,  31593),
            ('2021-05-02', 26923577,  58751),
            ('2021-05-03',  7182897,  14691),
            ('2021-05-03',  7250526,  14845),
            ('2021-05-03', 11264349,  21592),
            ...
            ('2021-06-10', 14703473,  73596),
            ('2021-06-10', 15322738,  78644),
            ('2021-06-10', 15341852,  78850),
            ('2021-06-10', 15345692,  78880),
            ('2021-06-10', 15400597,  79332),
            ('2021-06-10', 15400597,  79337),
            ('2021-06-10', 15400651,  79340),
            ('2021-06-10', 20048039, 104450),
            ('2021-06-10', 20227990, 107173),
            ('2021-06-10', 20245218, 107523)],
           names=['market_date', 'eid', None], length=263)

In [42]:
df = root_df.loc[root_df['market_date'] == '2021-06-08']
df = df.loc[df['poly'] == '+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)']
# df[['opp_id', 'eid', 't_time', 'side', '']]
def merge_synth_books(x):
    sb_start = synth_book.loc[synth_book['eid'] <= x.name, ['eid', 't_time']].values[-1]
    sb_start_eid = sb_start[0]
    sb_start_t_time = sb_start[1]
    sb_stop_t_time = sb_start_t_time + pd.Timedelta(500, unit='us')
    return synth_book.loc[(synth_book['eid'] >= sb_start_eid) & (synth_book['t_time'] <= sb_stop_t_time)]

df.groupby('eid').apply(merge_synth_books)

eid                        t_time  pid  bid_p_1  bid_q_1  ask_p_1  ask_q_1
eid                                                                                             
18431846 106877  18431846 2021-06-09 17:33:24.851077975    0     0.25        2     1.25       12
20885592 122501  20885592 2021-06-09 18:15:41.408121875    0     0.25        1     1.25       10
         122502  20885595 2021-06-09 18:15:41.408383977    0     0.25        1     1.25       10
         122503  20885598 2021-06-09 18:15:41.408391591    0     0.25        1     1.25       10
         122504  20885604 2021-06-09 18:15:41.408425547    0     0.25        1     1.25        9
         122505  20885610 2021-06-09 18:15:41.408430529    0     0.25        1     1.25        9
21015750 123794  21015750 2021-06-09 18:18:11.355843757    0     0.25        1     1.25        2
         123795  21015751 2021-06-09 18:18:11.356283467    0     0.25        2     1.25        2
         123796  21015752 2021-06-09 18:18:11.356283639    0     0.25        2     1.25        2
         123797  21015753 2021-06-09 18:18:11.356284011    0     0.25        2     1.25        2
         123798  21015755 2021-06-09 18:18:11.356302691    0     0.25        2     1.25        3
         123799  21015756 2021-06-09 18:18:11.356321277    0     0.25        1     1.25        3
         123800  21015757 2021-06-09 18:18:11.356338351    0     0.25        1     1.25        3
21081971 124395  21081971 2021-06-09 18:19:58.828540533    0     0.25        2     1.25        4
         124396  21081983 2021-06-09 18:19:58.829032889    0     0.25        2     1.25        4
         124397  21081984 2021-06-09 18:19:58.829035513    0     0.00        7     1.25        4
         124398  21081985 2021-06-09 18:19:58.829038277    0     0.00        5     1.25        4

In [25]:
fdf = df.loc[df['is_bf'] == False].drop(columns=['t_time', 'has_futures', 'min_days', 'max_days'])
fdf = fdf.loc[fdf['entry_ticks'] > 1]
qty_grps = fdf[['poly', 'entry_qty', 'entry_ticks', 'lsn_win_us']].groupby(['poly', 'entry_ticks'])
quantiles = [0.07, .1, .2, .3, .4, .5, .7, .8, .9]
SU.get_opp_duration_quantiles(qty_grps, quantiles, ['lsn_win_us']).sort_values(by=['n_opps'], ascending=False).head(100)

,n_opps,0.07_lsn_win_us,0.10_lsn_win_us,0.20_lsn_win_us,0.30_lsn_win_us,0.40_lsn_win_us,0.50_lsn_win_us,0.70_lsn_win_us,0.80_lsn_win_us,0.90_lsn_win_us
"(+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1), 2)",45,48.40,53.8,96.0,158.2,224.8,1620.0,983573.2,995676.4,999366.0
"(+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1), 2)",30,41.15,45.5,75.6,1597.6,14519.6,954720.5,994836.4,999263.6,999940.4
"(+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2), 2)",23,39.00,40.4,65.4,97.4,126.4,224.0,995522.2,997783.8,999339.0
"(+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1, 2)",16,598.10,1859.0,4266.0,31742.5,976841.0,987928.0,994560.5,998038.0,998998.5
"(+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1), 3)",14,325.60,522.4,1372.2,2592.4,197485.0,974080.5,994996.8,996388.4,997746.7
"(+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2), 2)",10,55.83,66.9,79.0,99.9,116.4,475710.0,996296.9,998330.8,998574.2
"(+ZSQ1 -(ZSQ1-ZSU1) -ZSU1, 2)",9,72.32,77.6,107.8,3675.4,196524.0,946616.0,987260.4,994133.6,995858.0
"(+ZSN1 -(ZSN1-ZSQ1) -ZSQ1, 2)",9,52.08,56.4,83.4,148.2,148424.4,741234.0,973155.0,982026.2,994487.0
"(+ZSN2 -(ZSN2-ZSX2) -ZSX2, 2)",8,23.19,29.7,40.2,43.1,50.8,59.0,843062.6,970487.4,995057.6
"(+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2), 2)",8,110.19,137.7,336.2,1236.4,5866.2,500796.5,995479.5,996195.4,997614.5
